Dask to find the context of SQL queries

In [1]:
from dask_sql import Context
import dask.dataframe as dd
c = Context()

Loading data from the Iris Dataset

In [2]:

df = dd.read_csv("https://datahub.io/machine-learning/iris/r/iris.csv")
# Register as Dask table
c.create_table("iris", df)

c.sql("""
    select * from iris
""")

,sepallength,sepalwidth,petallength,petalwidth,class
npartitions=1,,,,,
,float64,float64,float64,float64,object
,...,...,...,...,...


Training our clustering model

In [4]:
c.sql(
    """
        CREATE OR REPLACE MODEL clustering WITH (
            model_class = 'sklearn.cluster.KMeans',
            wrap_predict = True,
            n_clusters = 3
        ) AS (
            SELECT sepallength, sepalwidth, petallength, petalwidth
            FROM iris
        )
    """
)

c:\Users\Jack\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jack\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Jack\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(
c:\Users\Jack\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it 

In [5]:
# Show all models
c.sql(
    """
        SHOW MODELS
    """
)

,Models
npartitions=1,
0,object
0,...


In [6]:
# Describe the model 
c.sql(
    """
        DESCRIBE MODEL clustering
    """
)

,Params
npartitions=1,
estimator,object
transform_meta,...


In [8]:
# Prediction on each observation to a cluster

c.sql(
    """
        SELECT * FROM PREDICT (
            MODEL clustering,
            SELECT sepallength, sepalwidth, petallength, petalwidth
            FROM iris
            LIMIT 100
        )
    """
)

,sepallength,sepalwidth,petallength,petalwidth,target
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...


In [12]:
# Hyperparameter tuning
c.sql(
    """
       CREATE EXPERIMENT first_experiment WITH (
        model_class = 'sklearn.cluster.KMeans',
        experiment_class = 'GridSearchCV',
        tune_parameters = (n_clusters = ARRAY[2, 3, 4]),
        experiment_kwargs = (n_jobs = -1),
        target_column = 'target'
       ) AS (
        SELECT sepallength, sepalwidth, petallength, petalwidth, class AS target 
        FROM iris
        LIMIT 100
       )
    """
)

INFO:dask_sql.physical.rel.custom.create_experiment:{'n_clusters': [2, 3, 4]}


INFO:dask_sql.physical.rel.custom.create_experiment:{}
c:\Users\Jack\anaconda3\Lib\site-packages\dask\array\core.py:1713: FutureWarning: The `numpy.may_share_memory` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
c:\Users\Jack\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Jack\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,sepallength,sepalwidth,petallength,petalwidth,target,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,model_class
npartitions=1,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,int32,object
2,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
# EXPORT model

c.sql(
    """
    -- for pickle model serialization

    EXPORT MODEL clustering WITH (
        format='pickle',
        location='./tmp/clustering.pkl'
    )
    """
)

INFO:dask_sql.physical.rel.custom.export_model:Using model serde has pickle and model will be exported to ./tmp/clustering.pkl
